# Systeme de Recommandation V0
# Version 1 User



In [1]:
%pip install numpy
%pip install scipy
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:


import sys
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joans\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joans\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joans\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\joans\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
#recuperer la requete de filtrage et lutilsateur

user=sys.argv[1]
requete=sys.argv[2]


In [4]:
#Pour recuperer la listes des films faut avant recuperer tous les participants de la salle (liste de id de utilisateurs)
historique = "select id_show from regarder where id_us = " + str(user) + ";"

#recuperer la liste des films dans l'historique de l'utilisateur1(liste de id)
#recuperer la liste des films dans l'historique de l'utilisateur2(liste de id)
#recuperer la liste des films dans l'historique de l'utilisateur3(liste de id)
#recuperer la liste des films dans l'historique de l'utilisateur4(liste de id)


In [5]:

import pymysql
import pandas as pd

cnxn = pymysql.connect(user="nchoice", password="jenesaispas", host="localhost", database="nchoice")

print(cnxn)

In [6]:

#t = historique['id_show'].tolist()
t=("s3","s8","s10","s420","s888") #TEST
requete_historique = "SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ',') as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ',') as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ',') as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show IN {} GROUP BY show_new.id_show; ".format(t)

films_historiques = pd.read_sql(requete_historique, cnxn)
#films_filtres = pd.read_sql(requete, cnxn)

In [20]:
def list_similarity(list1, list2):
    if (list1 is None or list2 is None) or (len(list1) == 0 or len(list2) == 0):
        return 0
    set1 = set(map(tuple, list1))
    set2 = set(map(tuple, list2))

    intersection = set1.intersection(set2)
    union = set1.union(set2)

    similarity = len(intersection) / len(union) 
    return similarity


In [8]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    filtered_text = [word for word in text.lower().split() if word not in stop_words]
    
    # Perform stemming
    ps = PorterStemmer()
    stemmed_text = [ps.stem(word) for word in filtered_text]
    
    # Perform lemmatization
    lem = WordNetLemmatizer()
    lemmatized_text = [lem.lemmatize(word, "v") for word in stemmed_text]
    
    return " ".join(lemmatized_text)

def calculate_similarity(filtres, historique):
    genre_weight = 0.2
    director_weight = 0.15
    cast_weight = 0.20
    type_weight = 0.05
    description_weight = 0.3
    year_weight = 0.05
    country_weight = 0.05
    # Create a copy of the dataframes
    filtres_similarity = filtres.copy()
    historique_similarity = historique.copy()
    
    #Preprocess the  lists columns
    filtres_similarity["cast_ids"]=filtres_similarity["cast_ids"].str.split(",")
    filtres_similarity["director_ids"]=filtres_similarity["director_ids"].str.split(",")
    filtres_similarity["genre_ids"]=filtres_similarity["genre_ids"].str.split(",")
    filtres_similarity["country"]=filtres_similarity["country"].str.split(",")

    historique_similarity["cast_ids"]=historique_similarity["cast_ids"].str.split(",")
    historique_similarity["director_ids"]=historique_similarity["director_ids"].str.split(",")
    historique_similarity["genre_ids"]=historique_similarity["genre_ids"].str.split(",")
    historique_similarity["country"]=historique_similarity["country"].str.split(",")

    # Preprocess the description column
    filtres_similarity['description'] = filtres_similarity['description'].apply(preprocess_text)
    historique_similarity['description'] = historique_similarity['description'].apply(preprocess_text)

    #Acces rows of the filtered films dataframe and calculate the similarity between every filtered film and the films in the historial
    df_similarity = pd.DataFrame(columns=['id_show', 'similarity'])
    for index, row in filtres_similarity.iterrows():
        similarity=0
        for index2, row2 in historique_similarity.iterrows():
            if row['type']==row2['type']:
                similarity+=1*type_weight
            if row['year']==row2['year']:
                similarity+=1*year_weight
            similarity+=list_similarity(row['genre_ids'],row2['genre_ids'])*genre_weight
            similarity+=list_similarity(row['director_ids'],row2['director_ids'])*director_weight
            similarity+=list_similarity(row['cast_ids'],row2['cast_ids'])*cast_weight
            similarity+=list_similarity(row['country'],row2['country'])*country_weight
            vectorizer = CountVectorizer()
            # Transform the descriptions into vectors
            vectors = vectorizer.fit_transform([row["description"], row2["description"]])
            # Calculate the cosine similarity between the two vectors
            cosine_sim = cosine_similarity(vectors[0], vectors[1])[0][0]
            similarity+=cosine_sim*description_weight

        new_row={'id_show':row['id_show'],'similarity':similarity/len(historique_similarity)}
        df_similarity=df_similarity.append(new_row, ignore_index=True)

   

    return df_similarity.sort_values(by=['similarity'], ascending=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joans\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joans\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
#TEST
t=("s9","s812","s1010","s333","s784","s478","s897","s2588")
requete= "SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ',') as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ',') as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ',') as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show IN {} GROUP BY show_new.id_show; ".format(t)
films_filtres = pd.read_sql(requete, cnxn)
films_filtres
films_filtres = films_filtres.astype(str)
films_historiques = films_historiques.astype(str)
calculate_similarity(films_filtres, films_historiques)


,id_show,similarity
5,s812,0.100000
4,s784,0.098461
1,s2588,0.090676
0,s1010,0.090000
6,s897,0.061335
2,s333,0.054429
3,s478,0.049565
7,s9,0.030000


In [10]:
calculate_similarity(films_filtres, films_historiques)["id_show"].to_list()

['s812', 's784', 's2588', 's1010', 's897', 's333', 's478', 's9']

# TEST ZONE 

In [49]:
t=("s3") #TEST
requete_historique = "SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ',') as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ',') as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ',') as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show IN {} GROUP BY show_new.id_show; ".format(t)
requete_historique = 'SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ",") as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ",") as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ",") as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show = {} GROUP BY show_new.id_show;'

films_historiques = pd.read_sql(requete_historique, cnxn)
t = films_historiques['id_show'].tolist()
print(t)
len(t)

['s3']


1

In [54]:
#TEST
t=("s8")
requete = 'SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ",") as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ",") as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ",") as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show = {} GROUP BY show_new.id_show;'.format(t)
films_filtres = pd.read_sql(requete, cnxn)

t=("s8")
requete_historique = 'SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ",") as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ",") as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ",") as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show = {} GROUP BY show_new.id_show;'.format(t)

films_historiques4 = pd.read_sql(requete_historique, cnxn)
#films_filtres = pd.read_sql(requete, cnxn)
similarity4=calculate_similarity(films_filtres, films_historiques4)
print(similarity4)

DatabaseError: Execution failed on sql 'SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ",") as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ",") as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ",") as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show = {} GROUP BY show_new.id_show;': (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '} GROUP BY show_new.id_show' at line 1")

# Version plusieurs utilisateurs

# Simulate USER 1


In [26]:
#t = historique['id_show'].tolist()
t=("s3","s8","s10","s420","s887") #TEST
requete_historique = "SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ',') as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ',') as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ',') as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show IN {} GROUP BY show_new.id_show; ".format(t)

films_historiques1 = pd.read_sql(requete_historique, cnxn)
#films_filtres = pd.read_sql(requete, cnxn)
similarity1=calculate_similarity(films_filtres, films_historiques1)
print(similarity1)

  id_show  similarity
5    s812    0.100000
0   s1010    0.090000
4    s784    0.076571
6    s897    0.063820
3    s478    0.055676
1   s2588    0.054009
2    s333    0.040186
7      s9    0.040000


# Simulate USER 2

In [27]:
#t = historique['id_show'].tolist()
t=("s344","s84","s107","s4720","s787") #TEST
requete_historique = "SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ',') as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ',') as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ',') as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show IN {} GROUP BY show_new.id_show; ".format(t)

films_historiques2 = pd.read_sql(requete_historique, cnxn)
#films_filtres = pd.read_sql(requete, cnxn)
similarity2=calculate_similarity(films_filtres, films_historiques2)
print(similarity2)

  id_show  similarity
6    s897    0.097757
3    s478    0.080000
1   s2588    0.070305
4    s784    0.069919
7      s9    0.060000
5    s812    0.060000
2    s333    0.055000
0   s1010    0.043529


# Simulate USER 3

In [28]:
#t = historique['id_show'].tolist()
t=("s394","s844","s1057","s5120","s7487") #TEST
requete_historique = "SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ',') as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ',') as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ',') as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show IN {} GROUP BY show_new.id_show; ".format(t)

films_historiques3 = pd.read_sql(requete_historique, cnxn)
#films_filtres = pd.read_sql(requete, cnxn)
similarity3=calculate_similarity(films_filtres, films_historiques3)
print(similarity3)

  id_show  similarity
5    s812    0.090000
0   s1010    0.083638
3    s478    0.058975
4    s784    0.058149
6    s897    0.052544
2    s333    0.035000
1   s2588    0.030000
7      s9    0.030000


# Simulate USER 4

In [29]:
#t = historique['id_show'].tolist()
t=("s194","s804","s127","s680","s654") #TEST
requete_historique = "SELECT show_new.*, GROUP_CONCAT(DISTINCT jouer.id_cast SEPARATOR ',') as cast_ids, GROUP_CONCAT(DISTINCT produire.id_direc SEPARATOR ',') as director_ids,GROUP_CONCAT(DISTINCT etre.id_genre SEPARATOR ',') as genre_ids FROM show_new LEFT JOIN jouer ON jouer.id_show = show_new.id_show LEFT JOIN produire ON produire.id_show = show_new.id_show LEFT JOIN etre ON etre.id_show = show_new.id_show  WHERE show_new.id_show IN {} GROUP BY show_new.id_show; ".format(t)

films_historiques4 = pd.read_sql(requete_historique, cnxn)
#films_filtres = pd.read_sql(requete, cnxn)
similarity4=calculate_similarity(films_filtres, films_historiques4)
print(similarity4)

  id_show  similarity
3    s478    0.086000
6    s897    0.068944
7      s9    0.060000
5    s812    0.048757
1   s2588    0.047276
4    s784    0.038000
0   s1010    0.035000
2    s333    0.010000


In [35]:
merged_resultat = pd.DataFrame()
merged_resultat = merged_resultat.append(similarity1, ignore_index=True)
merged_resultat = merged_resultat.append(similarity2, ignore_index=True)
merged_resultat = merged_resultat.append(similarity3, ignore_index=True)
merged_resultat = merged_resultat.append(similarity4, ignore_index=True)
merged_resultat = merged_resultat.groupby('id_show').sum()
merged_resultat = merged_resultat.sort_values(by=['similarity'], ascending=False)
print(merged_resultat)

         similarity
id_show            
s9         0.680000
s12        0.550000
s812       0.250000
s1010      0.217167
s897       0.214122
s784       0.204640
s478       0.194651
s2588      0.154314
s333       0.130186
